##東森新聞

In [72]:
import requests 
from bs4 import BeautifulSoup
import cookielib
import json
import time 


urls_dic = {"綜合":"http://news.ebc.net.tw/apps/inc/newsList_General.inc",
            "社會":"http://news.ebc.net.tw/apps/inc/newsList_social.inc",
            "生活":"http://news.ebc.net.tw/apps/inc/newsList_Lifestyle.inc",
            "娛樂":"http://news.ebc.net.tw/apps/inc/newsList_Entertainment.inc",
            "國際":"http://news.ebc.net.tw/apps/inc/newsList_Politics.inc",
            "最多人瀏覽":"http://news.ebc.net.tw/apps/inc/newsList_MostClick.inc",
            "正妹":"http://news.ebc.net.tw/apps/inc/newsList_PrettyGirls.inc",
            "財經":"http://news.ebc.net.tw/apps/inc/newsList_Finance.inc",
            "兩岸":"http://news.ebc.net.tw/apps/inc/newsList_China.inc",
            "政治":"http://news.ebc.net.tw/apps/inc/newsList_Politics.inc"}

jar = cookielib.CookieJar()
headers = {'Host':'news.ebc.net.tw',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:38.0) Gecko/20100101 Firefox/38.0',
'Accept':'text/html, */*; q=0.01',
'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
'Accept-Encoding':'gzip, deflate',
'X-Requested-With':'XMLHttpRequest',
'Referer':'http://news.ebc.net.tw/apps/newsList.htm?menu=4',
'Cookie': jar,
'Connection':'keep-alive'}

# all news
L = []
for url in urls_dic:
    res = requests.get(urls_dic[url],headers = headers)
    res.encoding="utf-8"
    soup = BeautifulSoup(res.text)
    for block in soup.select(".newslist-box"):
        title=block.select('.newslist-title-txt')[0].text
        time=block.select('.gray_text')[0].text
        time= time.split("|")[0]
        aLink='http://news.ebc.net.tw/apps/' + block.select('a')[0]['href']
        L.append({'Category':url,'Title':title,'Link':aLink,'Time':time})
# pandas
import pandas as pd
df = pd.DataFrame(L)
df

# write in csv
f=open("ETTV.csv","w")
df.to_csv("ETTV.csv",encoding="utf-8")
f.close()

In [71]:
# content in all Link:
res = requests.get(url)
soup = BeautifulSoup(res.text)
soup.select("#divContent")

AttributeError: 'dict' object has no attribute 'split'

## 東森新聞雲

In [35]:
import requests
from bs4 import BeautifulSoup
import cookielib
import math
import re
from datetime import date,datetime, timedelta 
import time

# 各類新聞連結
def ETToday_links(days,sleep_time=0):
    #day = str(date.today()- timedelta(1))

    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                L.append({'Category':block.select('em')[0].text,'Title':title,'Link':aLink,'Time':day})
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Link':aLink,'Time':day})
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L)

    # write in csv
    f=open("ETToday_links.csv","w")
    df.to_csv("ETToday_links.csv",encoding="utf-8")
    f.close()



In [39]:
#各類新聞連結指定日期

def ETToday_links_day(day,sleep_time=0):
    urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                       }
    L = []
    for url in urls_dic:
        res = requests.get(urls_dic[url])
        res.encoding="utf-8"
        soup = BeautifulSoup(res.text)
        page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
        pageCount = int(math.ceil(float(page)/30))
        for block in soup.select("#all-news-list > h3"):
            title=block.select('a')[0].text
            aLink=block.select('a')[0]['href']
            L.append({'Category':block.select('em')[0].text,'Title':title,'Link':aLink,'Time':day})
            time.sleep(sleep_time)

        if pageCount > 1:
            for i in xrange(2,pageCount+1):
                url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                res = requests.get(url_1)
                res.encoding="utf-8"
                soup = BeautifulSoup(res.text)
                for block_1 in soup.select("#all-news-list > h3"):
                    title=block_1.select('a')[0].text
                    aLink=block_1.select('a')[0]['href']
                    L.append({'Category':block_1.select('em')[0].text,'Title':title,'Link':aLink,'Time':day})
                    time.sleep(sleep_time)
    # pandas
    import pandas as pd
    df = pd.DataFrame(L)

    # write in csv
    f=open("ETToday_links.csv","w")
    df.to_csv("ETToday_links"+"_day"+".csv",encoding="utf-8")
    f.close()


In [198]:
# 各新聞內容
import requests
from bs4 import BeautifulSoup
import re
url = "http://www.ettoday.net/news/20150613/520214.htm"
res=requests.get(url)
soup = BeautifulSoup(res.text)
#print soup.select('.title.clearfix')[0].text

    #標題
title_text = soup.select('.title.clearfix')[0].text.encode('utf-8')
if "《東森新聞雲》" in title_text:
    title = re.match("《東森新聞雲》(.*)", title_text).group(1)
else:
    title = title_text
    
    #內容
content = soup.select('.story > sectione > p')[1:]
len(content)
for i in xrange(0,len(content)):
    for con in content[i].select('img'):
        content[i].img.decompose()

    #記者
jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
journalist = (jour.split("記者")[1]).split("／")[0]


#扣除圖片敘述  -->沒辦法
#fb讚數  -->是粉絲團的
#fb commenter

#fb comment
#fb comment time


《東森新聞雲》掌握新媒體快速崛起　兩岸產學界驚豔


In [ ]:
# 完整版-直接輸入從今天算起要抓幾天 ETToday(days,sleep_time=0)
import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time

def connect(dbname,password,host,port=3306,user):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    db = MySQLdb.connect(host=host,user="root",port=port,passwd=password,db=dbname,charset="utf8")


def insert_news(category,title,newsDate,journalist,content,hit=None,url,source,correct=1,remark=None):
    cur = db.cursor()
    cur.execute(insert into news values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s),[category,title,newsDate,journalist,content,hit,url,source,correct,remark])
    db.commit()

def insert_comments(url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute(insert into comments values(%s,%s,%s,%s),[url,commenter,comment,remark])
    db.commit()
    
def close:
    db.close()

def ETToday(days,sleep_time=0):
    #day = str(date.today()- timedelta(1))
    
    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    f=open("ETToday.csv","w")
    df.to_csv("ETToday",encoding="utf-8")
    f.close()


In [ ]:
# 完整版-輸入單一日期或日期區間  ETToday_range(startday,endday,sleep_time=0) 

import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import calendar

def connect(dbname,password,host,port=3306,user):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    db = MySQLdb.connect(host=host,user="root",port=port,passwd=password,db=dbname,charset="utf8")


def insert_news(category,title,newsDate,journalist,content,hit=None,url,source,correct=1,remark=None):
    cur = db.cursor()
    cur.execute(insert into news values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s),[category,title,newsDate,journalist,content,hit,url,source,correct,remark])
    db.commit()

def insert_comments(url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute(insert into comments values(%s,%s,%s,%s),[url,commenter,comment,remark])
    db.commit()
    
def close:
    db.close()

def ETToday_range(startday,endday,sleep_time=0):
    #day = str(date.today()- timedelta(1))
    
    def convert_day(d):
        return datetime.datetime.strptime(d, '%Y-%m-%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)
    
    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y-%m-%d')))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    f=open("ETToday.csv","w")
    df.to_csv("ETToday",encoding="utf-8")
    f.close()


In [71]:
def ETToday_range(startday,endday,sleep_time=0):
    #day = str(date.today()- timedelta(1))
    
    def convert_day(d):
        return datetime.strptime(d, '%Y-%m-%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)
    
    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y-%m-%d')))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }
        print urls_dic["3C"]
ETToday_range('2015-01-01','2015-01-05',sleep_time=0)


http://www.ettoday.net/news/news-list-2015-01-05-20.htm
http://www.ettoday.net/news/news-list-2015-01-04-20.htm
http://www.ettoday.net/news/news-list-2015-01-03-20.htm
http://www.ettoday.net/news/news-list-2015-01-02-20.htm
http://www.ettoday.net/news/news-list-2015-01-01-20.htm


In [49]:
def convert_day(d):
        return datetime.strptime(d, '%Y-%m-%d')
d3=convert_day("2015-1-1")


TypeError: descriptor 'strftime' requires a 'datetime.date' object but received a 'str'

In [4]:
# run
ETToday_links(1)
ETToday_links_day('2015-06-14')
ETToday_links_C(1,1)

In [38]:
# insert sql
def connect(dbname,password,host,port=3306,user,):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    db = MySQLdb.connect(host=host,user="root",port=port,passwd=password,db=dbname,charset="utf8")

def insert_news(category,title,newsDate,journalist,content,hit=None,url,source,correct=1,remark=None):
    cur = db.cursor()
    cur.execute(insert into news values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s),[category,title,newsDate,journalist,content,hit,url,source,correct,remark])
    db.commit()

def insert_comments(url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute(insert into comments values(%s,%s,%s,%s),[url,commenter,comment,remark])
    db.commit()
    
def close:
    db.close()


,Category,Title,Link,Time
0,社會,快訊／七堵行政大樓停車場 男在車內燒炭亡,http://www.ettoday.net/news/20150614/520775.htm,2015-06-14
1,社會,勇消忍800度高溫無力跪地 吸幾口氧氣後...再回頭滅火,http://www.ettoday.net/news/20150614/520711.htm,2015-06-14
2,社會,快訊／不滿女友提分手 越男攀爬桃市府跳樓,http://www.ettoday.net/news/20150614/520708.htm,2015-06-14
3,社會,也是災情？情侶騎車遇暴雨 男自己穿雨衣女暴怒打他,http://www.ettoday.net/news/20150614/520737.htm,2015-06-14
4,社會,擔心遭撿屍…竹科女喝醉到超商休息 色男猥褻還洗劫,http://www.ettoday.net/news/20150614/520700.htm,2015-06-14
5,社會,替代役男割草猝死 母自責：教兒忍耐，笨媽反害死他,http://www.ettoday.net/news/20150614/520724.htm,2015-06-14
6,社會,快訊／騎單車被轉彎砂石車輾過 屏東鹽埔母女雙亡,http://www.ettoday.net/news/20150614/520690.htm,2015-06-14
7,社會,為平板推同學撞車？少年母：玩笑話，不小心拉扯,http://www.ettoday.net/news/20150614/520640.htm,2015-06-14
8,社會,美好回憶毀了...惡鄰占地拆房變賣 阿嬤老家變停車格,http://www.ettoday.net/news/20150614/520621.htm,2015-06-14
9,社會,2屁孩拒攔檢快閃鑽小巷 警連開28張紅單罰3萬,http://www.ettoday.net/news/20150614/520593.htm,2015-06-14


55

In [ ]:
stopword=["ETtoday","東森新聞雲","直擊／","快訊／"]

## 蘋果日報

In [99]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

driver = webdriver.Firefox()
driver.get("http://www.appledaily.com.tw/appledaily/article/headline/20150610/36599626/")
html = driver.page_source
content = str(html.encode('utf-8'))
soup = BeautifulSoup(content)
print soup.select('.articulum')[0].text

[<script src="http://twimg.edgesuite.net/www/js/html5.js"></script>, <script src="http://twimg.edgesuite.net/appledaily/images/js/jquery-1.7.2.min.js"></script>, <script src="http://twimg.edgesuite.net/appledaily/images/js/jquery.tools.dateinput.js"></script>, <script src="http://twimg.edgesuite.net/appledaily/images/js/jquery.lightbox-0.5.min-imgpv.js" type="text/javascript"></script>, <script src="http://twimg.edgesuite.net/appledaily/images/js/appledaily-detectmobilebrowser.js" type="text/javascript"></script>, <script type="text/javascript">
            function chartbeatvideo() {
                var mobile = (/iphone|ipad|ipod|android|blackberry|xbox|mini|windows\sce|palm/i.test(navigator.userAgent.toLowerCase()));
                if (!mobile) {
                   StrategyInterface.prototype.getTitle = function() {return '奇蹟 少女漂海21小時獲救';}
                   StrategyInterface.prototype.getVideoPath = function() {return 'http://twimg.edgesuite.net/video/20150610/9fd9b6d2eb0f77ac7616

In [5]:
import calendar
print calendar.monthrange(2011,1)

(5, 31)


NameError: name 'console' is not defined